In [1]:
import os
import requests
import json
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
MLFLOW_TRACKING_URI = "http://mlflow.mlops.svc.cluster.local:5000"
SERVICE_URL = "http://sales-profit-prediction.argo-workflow.svc.cluster.local:80/invocations"
MODEL_NAME = "sales_profit_model"

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [2]:
def get_production_preprocessing_id():
    client = MlflowClient()
    try:
        prod_models = client.get_latest_versions(MODEL_NAME, stages=["Production"])
        if not prod_models:
            raise RuntimeError(f"Kein Modell '{MODEL_NAME}' im Stage 'Production' gefunden.")
        
        latest_prod_model = prod_models[0]
        train_run_id = latest_prod_model.run_id
        print(f"ℹProduction Modell gefunden (Version {latest_prod_model.version}). Training Run ID: {train_run_id}")
        
    except Exception as e:
        print(f"Fehler beim Zugriff auf Model Registry: {e}")
        return None

    try:
        run = client.get_run(train_run_id)
        preproc_id = run.data.tags.get("preprocessing_run_id")
        
        if not preproc_id:
            raise RuntimeError("Tag 'preprocessing_run_id' fehlt im Training Run!")
            
        print(f"Preprocessing Run ID automatisch ermittelt: {preproc_id}")
        return preproc_id
        
    except Exception as e:
        print(f"Fehler beim Lesen der Run-Tags: {e}")
        return None

In [3]:
def send_request(params):

    preprocessing_run_id = get_production_preprocessing_id()
    if not preprocessing_run_id:
        return

    ## Feature Engineering
    df = pd.DataFrame([params])

    df["sales"] = df["sales"].astype(float)
    df["quantity"] = df["quantity"].astype(int)
    df["discount"] = df["discount"].astype(float)
    
    if "order_date" in df.columns:
        df["order_month"] = pd.to_datetime(df["order_date"]).dt.month
    
    df["original_price_per_unit"] = df["sales"] / (df["quantity"] * (1 - df["discount"]))
    df["markdown_amount"] = (df["original_price_per_unit"] * df["quantity"]) - df["sales"]

    ## Spalten filtern
    expected_cols = [
        "sub_category", "region", "segment", "order_month",
        "sales", "quantity", "original_price_per_unit", "markdown_amount"
    ]
    df_final = df[expected_cols]

    ## Preprocessor laden & Transformieren
    try:
        model_uri = f"runs:/{preprocessing_run_id}/preprocessing"
        preprocessor = mlflow.sklearn.load_model(model_uri)
        
        data_transformed = preprocessor.transform(df_final)
        
        if hasattr(data_transformed, "toarray"):
            data_transformed = data_transformed.toarray()
            
        data_list = data_transformed.tolist()
        
    except Exception as e:
        print(f"Fehler beim Laden/Transformieren (Preprocessing): {e}")
        return

    ## Datenanfrage senden
    payload = {"dataframe_split": {"data": data_list}}
    
    try:
        response = requests.post(
            SERVICE_URL,
            headers={"Content-Type": "application/json"},
            data=json.dumps(payload)
        )
        
        if response.status_code == 200:
            result = response.json()
            val = result[0] if isinstance(result, list) else result
            print("\n" + "="*40)
            print(f"VORHERGESAGTER PROFIT: {val:.2f} €")
            print("="*40)
            return val
        else:
            print(f"Server Fehler: {response.text}")
            
    except Exception as e:
        print(f"Verbindungsfehler: {e}")

In [ ]:
params = {
    "sales":        500.0,         # Umsatz der Transaktionszeile
    "quantity":     2,             # Menge
    "discount":     0.0,           # Rabatt: zwischen 0.0 und 1.0
    "sub_category": "Chairs",      # Muss vorhandene Kategorie sein: z.B. Phones, Chairs, Tables
    "region":       "West",        # South, West, Central, East
    "segment":      "Consumer",    # Consumer, Corporate, Home Office
    "order_date":   "2023-12-24"   # Datum (YYYY-MM-DD)
}